In [1]:
# Imports
# -----------

# required libraries
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
# tensorflow
import tensorflow as tf
from tensorflow import keras

import ipywidgets as widgets
from IPython.display import display

## Part 1 - Define functions to render controls

In [2]:
def create_slider_with_text_input(callback=None, label="slider", min=1, max=10, step=0.05):
    slider_input = widgets.FloatSlider(min=min, max=max, step=step, description=label, 
        continuous_update=False, layout={'width': '85%'})
    slider_input.style.handle_color = 'orange'
    text_input = widgets.FloatText(continuous_update=False, layout={'width': '10%'})
    widgets.link((text_input, 'value'), (slider_input, 'value'))
    if callback != None:
        slider_input.observe(callback, names='value')
        text_input.observe(callback, names='value')

    return slider_input, text_input


In [3]:
def create_controls(callback=None, output_section=None, show=False):
    slider_a, input_a = create_slider_with_text_input(label="ca0")
    slider_b, input_b = create_slider_with_text_input(label="beta")
    box_1 = widgets.HBox([slider_a, input_a])
    box_2 = widgets.HBox([slider_b, input_b])

    controls_box = widgets.VBox([box_1, box_2], 
        layout={'border': '1px solid black', 'margin': "15px", 'justify-content':"center"})

    if callback != None:
        out = widgets.interactive_output(callback, {'a': slider_a, 'b': slider_b})
    else:
        out = widgets.Output(layout={'border': '1px 1px 1px 1px solid black'})
    output =  out if output_section == None else output_section
    
    if show == True:
        display(controls_box, output)

    return controls_box, output

## Part 2 - Load model and predict data

In [4]:
#################
# Define Macros #
#################

# Define inputs and features
INPUT_PARAMS = ['a','b','time']
HAS_FAILED_FEATURE = ['fail']
STRESS_FEATURES = ['sx', 'sy', 'sz','sxy', 'sxz','syz']
DISPLACEMENT_FEATURES = ['u{axis}_{node}'.format(axis=a, node=i) for i in range(1,9) for a in ['x', 'y', 'z']]

# This macro determines if the "fail" feature should be 
# included in the FEATURES list of the model. 
INCLUDE_HAS_FAILED_FEATURE = False

STATS_PATH = 'stats/train_stats.pickle'
SAVED_MODEL_PATH = 'saved_models/model_1'

In [5]:
# Set the list of FEATURES that will be used in our model
if INCLUDE_HAS_FAILED_FEATURE == True:
    FEATURES = np.hstack((HAS_FAILED_FEATURE, DISPLACEMENT_FEATURES, STRESS_FEATURES))
else:
    FEATURES = np.hstack((DISPLACEMENT_FEATURES, STRESS_FEATURES))

In [6]:
##############
# Load model #
##############

model = tf.keras.models.load_model(SAVED_MODEL_PATH)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 60)                240       
_________________________________________________________________
dense_1 (Dense)              (None, 80)                4880      
_________________________________________________________________
dense_2 (Dense)              (None, 100)               8100      
_________________________________________________________________
dense_3 (Dense)              (None, 120)               12120     
_________________________________________________________________
dense_4 (Dense)              (None, 140)               16940     
_________________________________________________________________
dense_5 (Dense)              (None, 120)               16920     
_________________________________________________________________
dense_6 (Dense)              (None, 100)               1

In [7]:
###################################
# Load statistics and create norm #
###################################
stats = pd.read_pickle(STATS_PATH)

def norm(x):
    return (x - stats['mean']) / stats['std']

In [8]:
def predict_data(a, b):
    t_space = np.linspace(0,0.2,100)
    a_space = np.full(len(t_space), a)
    b_space = np.full(len(t_space), b)
    pred_inp = pd.DataFrame(data={'a': a_space, 'b': b_space, 'time': t_space})
    normed_pred = norm(pred_inp)
    predictions = pd.DataFrame(model.predict(normed_pred), columns=FEATURES)
    return pred_inp, predictions


In [9]:
def plot_f_curve(axis, pred_inp, predictions, feature, title=None):
    axis.scatter(pred_inp['time'], predictions[feature])
    axis.set_xlabel('time [s]')
    axis.set_ylabel('Predictions [{fe}]'.format(fe=feature))
    axis.set_title(title)

def plot_predictions(pred_inp, predictions, features):
    fs = np.array(np.copy(features))
    fs = np.array(np.split(fs, len(fs) / 3))
    fig, axs = plt.subplots(nrows=fs.shape[0], ncols=3, figsize=(15,30), dpi= 80)
    axs = axs.flatten() 
    for i, ax in enumerate(axs):
        plot_f_curve(ax, pred_inp, predictions, features[i], title="pred. " + features[i])
        
    fig.tight_layout()
    

In [10]:
def predict_and_plot(a, b):
    pred_inp, predictions = predict_data(a, b)
    plot_predictions(pred_inp, predictions, FEATURES)

In [11]:
controls_box, output = create_controls(predict_and_plot, show=True)

Output()